In [ ]:
from PIL import Image
import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import clip
from transformers import CLIPProcessor, CLIPModel

In [1]:
json_path = 'path to train_data.json'
image_path = 'path to training dataset'

In [2]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


device = "cuda:0" if torch.cuda.is_available() else "cpu" 

model, preprocess = clip.load("ViT-L/14", device=device, jit=False)

NameError: name 'CLIPModel' is not defined